### 1. RNN / LSTM의 한계

#### 1-1. RNN의 한계와 어텐션의 등장(복습)

* **RNN (Recurrent Neural Network) 복습**
    * 순차적 데이터(문장, 음악 등) 처리를 위해 이전 단계의 정보를 다음 단계로 전달하는 신경망.
    * **가중치 공유:** 모든 타임 스텝에서 동일한 가중치($W_{hx}$, $W_{hh}$ 등)를 적용하여 파라미터 효율성 및 일반화 성능 확보.
* **RNN의 한계**
    * **장기 의존성 문제 (Long-term dependency problem):**
        * 데이터가 길어질수록, 초반부의 정보가 후반부까지 잘 전달되지 않는 문제.
    * **기울기 소실 / 폭주 (Vanishing / Exploding Gradient):**
        * 학습 시 BPTT(Backpropagation Through Time) 방식을 사용.
        * 문장이 길어질수록 가중치 곱셈이 반복됨.
        * **Vanishing Gradient (기울기 소실):** 계산 값이 0에 수렴하여 초반부 데이터가 후반부에 반영되지 않음.
        * **Exploding Gradient (기울기 폭주):** 계산 값이 발산하여 초반부 데이터가 과도하게 반영됨.
* **LSTM (Long Short-Term Memory) 복습**
    * RNN의 Vanishing Gradient 문제 해결을 위해 등장 ("중요한 것은 오래 기억하고, 필요 없는 것은 잃어버리자").
    * **Cell state:** 가중치 곱을 거치지 않고 직선적으로 전달되는 정보 (메모장 역할).
    * **3가지 게이트:** Input, Forget, Output 게이트를 통해 정보 선택적 관리.
    * **LSTM의 한계:**
        * 3개의 게이트 운용으로 계산이 복잡하고 시간이 오래 걸림.
        * 긴 문장의 경우, 초반부 데이터가 많은 Forget Gate를 통과해야 하므로 여전히 Vanishing Gradient가 발생할 수 있음.
* **Attention의 등장**
    * **Seq2Seq의 한계 (병목 현상):**
        * Encoder가 입력 시퀀스의 모든 정보를 고정된 크기의 **Context Vector 하나**로 압축.
        * 입력 문장이 길어질수록 정보 소실 발생.
        * Decoder는 출력 시점과 상관없이 항상 동일한 Context Vector만 참고.
    * **Attention의 해결책:**
        * Decoder가 단어를 생성할 때마다, 입력 문장 전체를 다시 확인.
        * 현재 생성할 단어와 가장 관련 있는 정보들만 집중적으로 참고하여 **동적으로 새로운 Context Vector**를 생성.
    * **핵심 아이디어:** 결과를 만들 때, 입력값의 모든 부분을 동일하게 보지 않고, 현재 순간에 가장 중요한 부분에 더 높은 가중치를 부여.

#### 1-2. Attention is All You Need

* Google Brain (2017) 논문, **Transformer** 구조를 처음 발표 (Self-Attention 방식).
* **논문 배경 (RNN 한계):**
    * RNN 기반 모델들의 느린 학습 속도 (순차적 계산 구조).
    * 장기 의존성 문제.
* **핵심 제안:**
    * 순차적 계산 구조(Recurrence)와 Convolution을 아예 제거.
    * 인코더부터 디코더까지 오직 **Attention** 만을 사용하는 모델을 제안.
* **Transformer의 등장:**
    * RNN의 순환 구조를 완전히 제거하고, **Self-Attention**에 기반한 모델.
* **논문의 가장 중요한 발견 (트랜스포머의 핵심 원리):**
    * 성공적인 모델링에 Recurrence 및 Convolution이 필수가 아님을 증명.
    * 오직 Attention 만으로 기존 RNN 기반 모델들을 뛰어넘음.
    * 모든 단어들을 한 번에 계산할 수 있는 **병렬화**가 가능해짐.
* **파급 효과:**
    * 학습 속도를 획기적으로 단축, 훨씬 더 거대한 데이터 처리 가능.
    * LLM(BERT, GPT 등)의 직접적인 기반이 됨.
    * NLP를 넘어 컴퓨터 비전, 음성 처리 등 타 분야로 확장.

---

### 2. Self-Attention

* '문장 스스로가 문맥을 이해하는 과정'.
* 하나의 **동일한** 시퀀스(문장) 내에서 모든 단어끼리 서로의 관계를 계산.
* Query, Key, Value 쌍이 모두 동일한 시퀀스(입력 문장)에서 나옴.

#### 2-1. Query, Key, Value

* Attention이 관련 정보를 효과적으로 찾기 위해 입력 정보를 3가지 역할로 분할한 것.
* **Query (Q):**
    * '질문', '요청'의 역할.
    * 현재 초점을 맞추고 있는 대상 (예: 관계를 파악하려는 '나 자신' 단어).
* **Key (K):**
    * '이름표', '색인'의 역할.
    * 검색 대상이 되는 모든 정보 (예: 문장 내 모든 단어).
    * Q와의 관련도를 측정하는 데 사용됨.
* **Value (V):**
    * '실제 내용물', '실제 의미 정보'의 역할.
    * Key와 한 쌍으로 묶여 있음.
    * Q와 K의 관련도(어텐션 가중치)가 계산된 후, 이 가중치가 V에 곱해져 최종 결과에 반영됨.

#### 2-2. 기존 Cross-Attention vs. Self-Attention

| 구분 | 기존 어텐션 (Cross-Attention) | 셀프 어텐션 (Self-Attention) |
| :--- | :--- | :--- |
| **핵심 아이디어** | 번역할 문장을 보면서 단어 생성 | 문장 자체를 뜯어보며 내부 관계 파악 |
| **작동 방식** | 하나의 시퀀스가 **서로 다른** 시퀀스 참고 | **동일한** 시퀀스 내 단어끼리 관계 계산 |
| **참조 대상** | 서로 다른 시퀀스 (인코더 ↔ 디코더) | 동일한 시퀀스 (문장 내부 ↔ 문장 내부) |
| **주요 목적** | 인코더-디코더 연결, 정보 병목 해소 | 시퀀스 내부의 문맥 및 의존 관계 파악 |
| **Q, K, V 출처** | Q: 디코더<br>K, V: 인코더 | Q, K, V: 모두 동일한 시퀀스 |
| **병렬 처리** | 불가능 (순차적) | 가능 |

#### 2-3. Scaled Dot-Product Attention

* 트랜스포머에서 사용하는 표준 어텐션 계산 공식.
* **Dot-Product Attention의 문제:**
    * 단순히 Q와 K를 내적(Dot Product)하여 유사도(Attention Score)를 계산.
    * 벡터의 차원($d_k$)이 커질수록 내적 값이 과도하게 커져, Softmax 함수 통과 시 기울기가 0에 가까워지는 Vanishing Gradient 문제 발생.
* **Scaled Dot-Product Attention:**
    * 위 문제를 해결하기 위해, 계산된 내적 값을 $\sqrt{d_K}$ (키 벡터 차원의 제곱근)로 나누어 스케일링(Scaling)함.
    * 스케일링을 통해 값이 과도하게 증가하는 것을 방지하고 안정적인 분포를 유지하여 학습을 안정화.
* **수식:**
    $$Attention(Q,K,V)=softmax(\frac{QK^T}{\sqrt{d_K}})V$$

---

### 3. Multi-Head Attention

* Transformer 아키텍처의 핵심 연산 단위. 인코더와 디코더의 여러 위치에서 사용됨.

#### 3-1. Why Multi-Head Attention? (핵심 원리 및 장점)

* **Single-Head Attention의 한계:**
    * 하나의 가중치 행렬 세트($W^Q, W^K, W^V$)만 학습하고 사용.
    * 문법적 관계, 의미적 관계, 위치 관계 등 모든 종류의 정보를 하나의 어텐션 분포에 담으려 함.
    * 결국 각 단어 간의 중요한 관계가 다른 정보와 평균화되어 약해질 수 있음.
* **Multi-Head Attention (핵심 원리):**
    * 한 단어와 다른 단어 간의 관계를 여러 차원(관점)으로 나누어 **병렬**로 학습.
    * $d_{model}$ 차원(예: 512)의 입력을 $h$개(예: 8개)의 헤드로 분할.
    * $h$개의 헤드가 각각 **서로 다른** 가중치 행렬을 가지고 어텐션을 독립적으로 수행 (예: 8개의 64차원 어텐션).
* **장점:**
    * 모델이 여러 관점에서 정보를 동시에 학습 가능.
    * 각 헤드는 특정 유형의 관계(예: 문법적 관계, 의미적 연결)에 집중하도록 특화될 수 있음.

#### 3-2. Multi-Head Attention 작동 방식

* **1. 분할 (Projection):**
    * 입력 벡터(예: 512차원)에 $h$개의 서로 다른 가중치 행렬 세트($W_i^Q, W_i^K, W_i^V$)를 곱함.
    * $h$개의 독립적인 Q, K, V 세트(예: 64차원)를 생성.
* **2. 병렬 어텐션 계산:**
    * $h$개의 헤드가 각각 독립적으로 Scaled Dot-Product Attention을 병렬 수행.
    * $h$개의 결과 벡터(Attention Output, 예: 64차원)가 나옴.
* **3. 결합 및 최종 투영:**
    * $h$개의 결과 벡터를 모두 이어 붙임(Concatenate) (예: 8 x 64 = 512차원).
    * 이 결합된 벡터에 최종 가중치 행렬($W^O$)을 곱하여 Multi-Head Attention의 최종 출력 벡터(예: 512차원)를 생성.

---

### 4. Transformer 전체 아키텍처

#### 4-1. 트랜스포머의 전처리 단계

* 텍스트를 모델이 연산할 수 있는 벡터 형태로 변환하는 단계.
* **1) 토큰화 (Tokenization):**
    * 입력 텍스트(문장)를 모델이 처리할 수 있는 단위(Token)로 분할.
    * 토큰은 단어, 문장 부호 또는 더 작은 단위(Subword)일 수 있음.
* **2) 임베딩 (Embedding):**
    * 각 토큰을 고유한 숫자 벡터(Embedding Vector)로 변환.
    * 이 벡터는 단어의 의미론적 속성을 포함함 (유사한 단어는 벡터 공간에서 가까운 위치).
* **3) Positional Encoding:**
    * **필요성:**
        * 트랜스포머는 RNN과 달리 순차적 구조가 아니며, 모든 단어를 병렬로 한 번에 처리함.
        * 따라서 모델은 단어의 '순서' 정보를 알지 못함.
    * **역할:**
        * 단어의 상대적 또는 절대적 위치(순서) 정보를 임베딩 벡터에 추가함.
    * **작동 방식:**
        * 각 단어의 위치($pos$)마다 고유한 값을 가지는 위치 벡터(Positional Encoding Vector)를 생성.
        * 이 위치 벡터를 단어의 임베딩 벡터에 **합산** (Element-wise Addition).
        * 이를 통해, 같은 단어라도 문장 내 위치가 다르면 최종적으로 다른 벡터 값을 갖게 됨.
    * **생성 방법 (Sine/Cosine 함수):**
        $$PE_{(pos, 2i)} = \sin(\frac{pos}{10000^{2i/d_{model}}})$$
        $$PE_{(pos, 2i+1)} = \cos(\frac{pos}{10000^{2i/d_{model}}})$$

#### 4-2. 트랜스포머의 Encoder와 Decoder

* **1) Encoder (인코더):**
    * **역할:** 입력 문장을 이해하고 요약된 의미 벡터(Context Vector)로 변환.
    * **구조:** N개의 동일한 인코더 레이어가 쌓여 있음.
    * **각 인코더 레이어의 구성 (2개의 하위 레이어):**
        1.  **Multi-Head Attention (Self-Attention):** 입력 문장 내 단어 간의 관계를 파악.
        2.  **Feed-Forward Layer (FFN):** 비선형 변환 적용 (2층 MLP 구조).
    * **공통 모듈:**
        * **Residual connections (잔차 연결):** 각 하위 레이어의 입력을 출력에 더함 ($X + SubLayer(X)$). 층이 깊어져도 학습이 잘 되도록 함 (기울기 소실 완화).
        * **Layer Normalization (층 정규화):** 각 하위 레이어를 통과한 후 적용되어 학습을 안정화하고 속도를 높임.
* **2) Decoder (디코더):**
    * **역할:** 인코더가 분석한 입력 문장의 의미 벡터를 받아서 출력 문장을 순차적으로 생성.
    * **구조:** N개의 동일한 디코더 레이어가 쌓여 있음.
    * **각 디코더 레이어의 구성 (3개의 하위 레이어):**
        1.  **Masked Multi-Head Attention:**
            * 디코더의 출력 문장 내부의 관계를 파악 (Self-Attention).
            * **Masking (Look-Ahead Mask):** 미래 시점의 단어 정보를 참고하지 못하도록 마스크(mask)를 적용. 다음 단어를 예측할 때 정답을 엿보지 못하게 함.
        2.  **Encoder-Decoder Attention (Cross-Attention):**
            * 디코더가 인코더의 출력(Context)을 참고하는 과정.
            * **Query (Q):** 디코더의 이전 하위 레이어 출력.
            * **Key (K), Value (V):** **인코더의 최종 출력 (Context Vector)**.
            * 이를 통해 디코더가 입력 문장 중 어떤 부분에 주목해야 하는지 학습.
        3.  **Feed-Forward Layer (FFN)**
    * **공통 모듈:** 인코더와 동일하게 각 하위 레이어마다 Residual Connection과 Layer Normalization이 적용됨.

#### 4-3. 전체 모델 및 데이터 흐름

* **1. 입력 준비 (인코더 입력):**
    * 입력 문장 → 토큰화 → 임베딩.
    * 임베딩 벡터 + Positional Encoding 벡터 = 인코더의 최종 입력.
* **2. 인코더 (N회 반복):**
    * 입력 벡터가 [Multi-Head Self-Attention → Add & Norm → FFN → Add & Norm]을 N번 통과.
    * 입력 문장의 문맥적 의미를 모두 반영한 **Context Vectors** 집합 생성 ($K_{enc}, V_{enc}$).
* **3. 디코더 (한 단어씩 순차적 생성):**
    * **입력:** <start> 토큰 또는 이전에 생성된 단어. (임베딩 + 포지셔널 인코딩)
    * **1단계 (Masked Self-Attention):** 이미 생성된 출력 단어들 간의 관계 계산. (Add & Norm)
    * **2단계 (Encoder-Decoder Attention):** 1단계의 결과(Q)가 인코더의 $K_{enc}, V_{enc}$를 참조하여 입력 문장의 어떤 부분에 집중할지 결정. (Add & Norm)
    * **3단계 (FFN):** 2단계의 결과를 비선형 변환. (Add & Norm)
    * **4단계 (Linear & Softmax):** 3단계의 최종 벡터를 어휘 사전 크기로 변환 후, 확률이 가장 높은 단어를 다음 단어로 선택.
    * <end> 토큰이 생성될 때까지 위 과정을 반복.